### Define Helper Functions

In [1]:
from os import listdir
from os.path import isfile, join
import pandas as pd
from datetime import datetime,date,timedelta
import json 
from collections import deque
import pickle
import functools
import time

import multiprocessing  

import re

def matchLineIndex(s):
    ### Convert GLD notation to GeoJson Notation
    integer_list=re.findall(r'\d+', s)    
    return 'line'+integer_list[0]+"to"+integer_list[1]

def disaggMap(dataFrame,nodes_to_loads,parseCase,meters):
    dataFrames=[]
    total=0
    for elem in meters:
        #print(len(dataFrame))
        frame=dataFrame[dataFrame['month']==parseCase['month']].copy(deep=True)
        frame['node']=elem
        counts=nodes_to_loads[nodes_to_loads['name']==elem]
        #print(counts)

        for index,row in frame.iterrows():
            for typ in counts.keys():
                if row['type']==typ:
                    #print(list(frame.columns.difference(['node','hour_of_day','month','type'])))
                    for id in list(frame.columns.difference(['node','hour_of_day','month','type'])):
                        frame.loc[index, id]=float(row[id])*float(counts[typ])
        dataFrames.append(frame)
    finalFrame=pd.concat(dataFrames)
    return(finalFrame)


def getPickles(mypath):
    pickleFiles={}
    for f in set(listdir(mypath)):
        if isfile(join(mypath, f)) and (f[-6:]=="pickle"):
            path=join(mypath, f)
            resourceType=f.replace('.pickle','')
            pickleFiles[resourceType]=path
    return pickleFiles

def parseLoadNodeNames(name):
    if name[0:4]=='load':
        number=name.split('load:')
        nodeType='load_'
    if name[0:4]=='node':
        number=name.split('node:')
        nodeType='node_'
    return nodeType+str(number[1])    
from collections import defaultdict

def children(token, tree):
    "returns a list of every child"
    visited = set()
    to_crawl = deque([token])
    while to_crawl:
        current = to_crawl.popleft()
        if current in visited:
            continue
        visited.add(current)
        node_children = set(tree[current])
        to_crawl.extend(node_children - visited)
    return list(visited)

def addDashtoNodeName(nodename):
    nodename=nodename.split('node')
    return('node_'+nodename[1])

def addDashtoLoadName(loadname):
    loadname=loadname.split('load')
    return('load_'+loadname[1])

def columnFormat(columnName):
    columnName=columnName.lower()
    if columnName[0:4]=='node':
        columnName=addDashtoNodeName(columnName)
    if columnName[0:4]=='load':
        columnName=addDashtoLoadName(columnName)
    return columnName

def monthMapper(month):
    date = datetime(2000, month, 1)  # We need a dateobject with the proper month
    return date.strftime('%B').lower()

def parseVoltages(voltage_files):
    voltage_dfs={}
    voltage_dfs_flattened={}
    for voltage_name,filename in voltage_files.items():
        voltage_dfs[voltage_name]=pd.read_csv(filename,skiprows=list(range(0,8)),parse_dates=['# timestamp'])

    for voltage_name,df in voltage_dfs.items():
        df.columns = [columnFormat(col_name) for col_name in df.columns]
        df['# timestamp'] = df['# timestamp'].map(lambda x: x.tz_localize('UTC').tz_convert('US/Pacific'))
        df['month'] = df['# timestamp'].map(lambda x: monthMapper(x.month))
        df['hour_of_day'] = df['# timestamp'].map(lambda x: x.hour)
        df['day_of_month'] = df['# timestamp'].map(lambda x: x.day)
        df['timezone']= df['# timestamp'].map(lambda x: x.tz)
        ### Get all the node names
        nodeNames=[]
        for columnName in df.columns:
            if columnName[0:4]=='node':
                nodeNames.append(columnName)
            if columnName[0:4]=='load':
                nodeNames.append(columnName)

        df=pd.melt(df, id_vars=['month','hour_of_day','day_of_month'],var_name='node',value_vars=nodeNames, value_name=voltage_name+'_magnitude')
        df[voltage_name+'_magnitude']=df[voltage_name+'_magnitude'].map(lambda x:abs(complex(x.replace('i','j'))))


        voltage_dfs_flattened[voltage_name]=df
    voltageData = functools.reduce(lambda left,right: pd.merge(left,right,on=['month', 'day_of_month', 'hour_of_day','node']), voltage_dfs_flattened.values())
    
    return(voltageData)

def tryconvertreal(value):
        try:
            S=complex(value.replace('i','j'))
            return S.real
        except:
            return value

        
def tryconvertimag(value):
        try:
            S=complex(value.replace('i','j'))
            return S.imag
        except:
            return value
        
def tryconvertmag(value):
        try:
            return abs(complex(value.replace('i','j')))
        except:
            return value
        
def parsePowerFlow(power_files):
    power_dfs={}
    power_dfs_flattened={}
    for power_name,filename in power_files.items():
        power_dfs[power_name]=pd.read_csv(filename,skiprows=list(range(0,8)),parse_dates=['# timestamp'])

    for power_name,df in power_dfs.items():
        df['# timestamp'] = df['# timestamp'].map(lambda x: x.tz_localize('UTC').tz_convert('US/Pacific'))
        df['month'] = df['# timestamp'].map(lambda x: monthMapper(x.month))
        df['hour_of_day'] = df['# timestamp'].map(lambda x: x.hour)
        df['day_of_month'] = df['# timestamp'].map(lambda x: x.day)
        df['timezone']= df['# timestamp'].map(lambda x: x.tz)
        
        lineNames=[]
        replaceCols=[]
        for lineName in df.columns:
            if lineName[0]=='O':
                geoJsonLineName=matchLineIndex(lineName)
                lineNames.append(geoJsonLineName)
                replaceCols.append(geoJsonLineName)
            else:
                replaceCols.append(lineName)

        df.columns=replaceCols
        df=pd.melt(df, id_vars=['month','hour_of_day','day_of_month'],var_name='line',value_vars=lineNames, value_name=power_name+'_magnitude')
        df[power_name+'_P']=df[power_name+'_magnitude'].map(lambda x:tryconvertreal(x))
        df[power_name+'_Q']=df[power_name+'_magnitude'].map(lambda x:tryconvertimag(x))
        df[power_name+'_magnitude']=df[power_name+'_magnitude'].map(lambda x:tryconvertmag(x))


        power_dfs_flattened[power_name]=df
    powerData = functools.reduce(lambda left,right: pd.merge(left,right,on=['month', 'day_of_month', 'hour_of_day','line']), power_dfs_flattened.values())

    return(df,powerData)


def parseTransmission(transmission_files):
    # # # Line data 
    # # # Dates,Pfrom(kW),Qfrom(KVar),Pto(kW)
    # # # Node data 
    # # # Dates,Vm(kV),Va(deg),Pl(kW),Ql(KVar)

    transmission_dfs={}
    transmission_dfs_flattened={}
    for transmission_name,filename in transmission_files.items():
        transmission_dfs[transmission_name]=pd.read_csv(filename,skiprows=list(range(0,2)),parse_dates=['Dates'])

    for transmission_name, df in transmission_dfs.items():
        df['Dates'] = df['Dates'].map(lambda x: x.tz_localize('UTC').tz_convert('US/Pacific'))
        df['month'] = df['Dates'].map(lambda x: monthMapper(x.month))
        df['hour_of_day'] = df['Dates'].map(lambda x: x.hour)
        df['day_of_month'] = df['Dates'].map(lambda x: x.day)
        df['timezone']= df['Dates'].map(lambda x: x.tz)

        transmission_dfs_flattened[transmission_name]=df
    transmissionData = functools.reduce(lambda left,right: pd.merge(left,right,on=['month', 'day_of_month', 'hour_of_day']), transmission_dfs_flattened.values())

    return(df,transmissionData)




In [2]:
def parseDERCAM(filenames,weekend,verbose):
    st_time=time.time()
    print("*********Parsing DERCAM")
    ### PARSE DERCAM FILES
    ## Flatten files to facilitate the use of crossfilter and dc.js
    if weekend:
        keys=['weekend']
    else:
        keys=['week']
    dataFrames=[]
    for resourceType,filename in filenames.items():
        try:
            with open(filename, 'rb') as input_file:
                data = pickle.load(input_file)
            ct=0
            if verbose:
                print('These exist')
                print([val for val in set(list(data.keys())) if val in set(dataKeys)])
            if verbose:
                print('These are missing')
                print(set(list(data.keys())).symmetric_difference(set(dataKeys)))
            for elem in dataKeys:
                try:
                    for key in keys:#data[elem].keys():
                        #print(filename,elem,key)
                        #print(data[elem].keys())
                        dummy=data[elem][key]
                        #ataFrame['type']=key
                        #ataFrame['variable']=elem
                        dummy=pd.melt(dummy, id_vars=['month'],var_name='hour_of_day',value_name=elem)
                        dummy[elem] = dummy[elem].astype(float)
                        if ct==0:
                            dataFrame=dummy
                        if ct>0:
                            dataFrame=pd.merge(dataFrame,dummy,on=['month','hour_of_day'])
                        ct=ct+1
                except:
                    if verbose:
                        print('failed for '+filename,elem)
                    continue
            dataFrame['type']=resourceType
            if 'pv_inst' in set(list(data.keys())):
                dataFrame['pv_inst']=float(data['pv_inst'])
            if 'storage_inst' in set(list(data.keys())):
                dataFrame['storage_inst']=float(data['storage_inst'])
            if 'chp_inst' in set(list(data.keys())):
                dataFrame['chp_inst']=float(data['chp_inst'])
            dataFrames.append(dataFrame)
        except:
            if verbose:
                print("Couldn't process "+filename)

    dataFrame=pd.concat(dataFrames)
    dataFrame=dataFrame.fillna(value=0)
    dataFrame=dataFrame.reset_index(drop=True)
    print("Took ",time.time()-st_time)

    return dataFrame

def parseTopology(dataFrame):
    ### PARSE TOPOLOGY
    st_time=time.time()
    print("*********Parsing Topology")
    with open('ieee123.json') as data_file:    
        data = json.load(data_file)
    fromToDict=defaultdict(list)
    for elem in data:
        if 'overhead_line:' in elem['selector']:
            fromToDict[parseLoadNodeNames(elem["properties"]['from'])].append(parseLoadNodeNames(elem["properties"]['to']))


    nodeChildrenDict=defaultdict(list)
    for node in set(daily_node_voltage['node']):
        for elem in children(node,fromToDict):
            if elem[0:4]=='load':
                nodeChildrenDict[node].append(elem)

    nodes_to_loads=pd.read_csv(dercam_scenario_path+"/disaggregation.csv")
    nodes_to_loads['name']=nodes_to_loads['name'].apply(lambda x:x.replace('Load','load_') )


    for node,node_list in nodeChildrenDict.items():
        dummy=nodes_to_loads[
            nodes_to_loads['name'].isin(
                node_list)].sum(axis=0)
        dummy['name']=node
        nodes_to_loads=nodes_to_loads.append(dummy, ignore_index=True)
    ## Sum up everything and create node and load consumption patterns
    meters=list(nodes_to_loads['name'])
    print("Took ",time.time()-st_time)
    st_time=st_time2
    
    return meters,nodes_to_loads



In [3]:
def getGLDParseList(gridlabd_results_path,scenario_name,location_name):
    parseList=[]
    ### FOR EACH MONTH 
    for i in range(1,13):
        mydate = date(2016,i,1)
        fileLocation="/Month"+"%02d" % i +"/output/"
        gridlabd_scenario_path=gridlabd_results_path+scenario_name+"/Models/"+location_name+fileLocation
        gld=pd.read_csv(gridlabd_scenario_path+'/nodes_voltage_A.csv',skiprows=list(range(0,8)),parse_dates=['# timestamp'])
        datetimes=gld['# timestamp'].map(lambda x: x.tz_localize('UTC').tz_convert('US/Pacific').date())
        dayWeekday={}
        weekend=False
        weekday=False
        for elem in sorted(set(datetimes)):
            if elem.weekday()<5:
                weekday=elem.day
            else: 
                weekend=elem.day

            if weekend & weekday:
                break
        parseList.append({'month':mydate.strftime("%B").lower(),'filename':location_name+fileLocation,'weekday':weekday,'date':mydate+timedelta(days=weekday-1)})
        parseList.append({'month':mydate.strftime("%B").lower(),'filename':location_name+fileLocation,'weekend':weekend,'date':mydate+timedelta(days=weekend-1)})
    return parseList


### Parse Node Level Results from DERCAM, GridlabD and Topology Files for Distribution System View

In [8]:
### SET PATH FOR DERCAM RESULTS
# /Users/emrecan/Work_Git/GMLC/GMLC_Results/BC/output/IEEE-123-PGE/gmlc/buildings


### SET PATH FOR GRIDLABD RESULTS
# /Users/emrecan/Work_Git/GMLC/GMLC_Results_TD/BC/Models/PGE-157/Month02/output

gridlabd_results_path="~/GMLC_Results_TD/"
dercam_results_directory="~/GMLC_Results/"
transmission_results_path="~/GMLC/2018_GMLC_3point_test/"
scenario_names=["BC","PV","PV+0.7STO+CHP","PV+STO","PV+STO+CHP", "PV+STO+CHP+NM"]

### descriptor.json location

descriptor_file_path="~/GMLCDjangoProject/MapApp/static/MapApp/data/descriptor.json"

analyzed_data_path='../GMLCDjangoProject/MapApp/static/MapApp/data/' 

import json
from pprint import pprint

with open(descriptor_file_path) as data_file:    
    data = json.load(data_file)
pprint(data)

{'distribution': [{'name': 'PGE',
                   'scenarios': [{'name': 'BC', 'value': 'scenario1'},
                                 {'name': 'PV', 'value': 'scenario2'},
                                 {'name': 'PV+0.7STO+CHP',
                                  'value': 'scenario3'},
                                 {'name': 'PV+STO', 'value': 'scenario4'},
                                 {'name': 'PV+STO+CHP', 'value': 'scenario5'},
                                 {'name': 'PV+STO+CHP+NM',
                                  'value': 'scenario6'}],
                   'value': 'pge'},
                  {'name': 'SCE',
                   'scenarios': [{'name': 'BC', 'value': 'scenario7'},
                                 {'name': 'PV', 'value': 'scenario8'},
                                 {'name': 'PV+0.7STO+CHP',
                                  'value': 'scenario9'},
                                 {'name': 'PV+STO', 'value': 'scenario10'},
                                 

In [30]:
dataKeys=['hourly_pv_self_consumpt',
          'hourly_pv_export',
          'elec_stored_stationnary_batteries',
          'purchase_from_utility',
          'elec_provided_by_stationnary_battery_charging_after_eff','pv_inst','storage_inst','chp_inst']
verbose=False

descriptor={}
### DIVIDE BELOW PARSELIST CONSTRUCTION TO A SEPERATE FUNCTION
descriptor['distribution']=[]
scenario_number=0
for location_name in ['PGE','SCE','SDGE']:
    descriptor_dict={}
    scenario_descriptor_list=[]
    for scenario_name in scenario_names:
        scenario_descriptor={}
        scenario_number=scenario_number+1
        save_final_data_file=analyzed_data_path+location_name+'/scenario'+str(scenario_number)+'.csv'
        parseList=getGLDParseList(gridlabd_results_path,scenario_name,location_name)
        print(parseList)
        merge_list=[]

        for parseCase in parseList:

            dercam_scenario_path=dercam_results_directory+scenario_name+"/output/"+location_name+"/gmlc/buildings"
            gridlabd_scenario_path=gridlabd_results_path+scenario_name+"/Models/"+parseCase['filename']

            filenames=getPickles(dercam_scenario_path)
            print(dercam_scenario_path)

            voltage_files={'voltage_a':gridlabd_scenario_path+'/nodes_voltage_A.csv',
                           'voltage_b':gridlabd_scenario_path+'/nodes_voltage_B.csv',
                           'voltage_c':gridlabd_scenario_path+'/nodes_voltage_C.csv',}

            load_voltages={'voltage_a':gridlabd_scenario_path+'/loads_voltage_A.csv',
                           'voltage_b':gridlabd_scenario_path+'/loads_voltage_B.csv',
                           'voltage_c':gridlabd_scenario_path+'/loads_voltage_C.csv'}

            #### PARSE GRIDLABD
            print("*********Parsing GridlabD")
            st_time=time.time()
            voltageData=parseVoltages(voltage_files)
            loadVoltageData=parseVoltages(load_voltages)
            if 'weekend' in parseCase.keys():
                if verbose:
                    print('Parsing Weekend')
                day_of_month=parseCase['weekend']
                weekend=True
            else:
                day_of_month=parseCase['weekday']
                weekend=False

            daily_node_voltage=voltageData[voltageData['month']==parseCase['month']][voltageData['day_of_month']==day_of_month][["month","hour_of_day","node",
                                                                                                             "voltage_b_magnitude","voltage_c_magnitude" ,"voltage_a_magnitude"]]
            daily_load_voltage=loadVoltageData[loadVoltageData['month']==parseCase['month']][loadVoltageData['day_of_month']==day_of_month][["month","hour_of_day","node",
                                                                                                              "voltage_b_magnitude","voltage_c_magnitude" ,"voltage_a_magnitude"]]
            st_time2=time.time()
            print("Took ",st_time2-st_time)
            st_time=st_time2

            dataFrame=parseDERCAM(filenames,weekend,verbose)
            [meters,nodes_to_loads]=parseTopology(dataFrame)
            
            print("*********Parsing Disagg")

            finalFrame=disaggMap(dataFrame,nodes_to_loads,parseCase,meters)

            st_time2=time.time()
            print("Took ",st_time2-st_time)
            st_time=st_time2


            ## Down select DERCAM data to match the month
            dercamData=finalFrame[finalFrame['month']==parseCase['month']]
            dercamData['hour_of_day']=dercamData['hour_of_day'].apply(lambda x:x-1)
            ## Stich everything GLD and DERCAM all together
            voltageFrame=daily_load_voltage.append(daily_node_voltage)
            merge_test=dercamData.merge(voltageFrame,on=['month', 'hour_of_day','node'],how='left')
            if weekend:
                merge_test['daytype']='weekend'
            else:
                merge_test['daytype']='weekday'

            
            merge_list.append(merge_test)
            
        scenario_descriptor['name']=scenario_name
        scenario_descriptor['value']="scenario"+str(scenario_number)
        scenario_descriptor_list.append(scenario_descriptor)


        print('Saving file to '+save_final_data_file)
        merge_test_df=pd.concat(merge_list)
        merge_test_df.reset_index(drop=True).to_csv(save_final_data_file)

    descriptor_dict['scenarios']=scenario_descriptor_list
    descriptor_dict['name']=location_name
    descriptor_dict['value']=location_name
    descriptor['distribution'].append(descriptor_dict)





[{'month': 'january', 'filename': 'PGE/Month01/output/', 'weekday': 1}, {'month': 'january', 'filename': 'PGE/Month01/output/', 'weekend': 3}, {'month': 'february', 'filename': 'PGE/Month02/output/', 'weekday': 5}, {'month': 'february', 'filename': 'PGE/Month02/output/', 'weekend': 6}, {'month': 'march', 'filename': 'PGE/Month03/output/', 'weekday': 4}, {'month': 'march', 'filename': 'PGE/Month03/output/', 'weekend': 5}, {'month': 'april', 'filename': 'PGE/Month04/output/', 'weekday': 1}, {'month': 'april', 'filename': 'PGE/Month04/output/', 'weekend': 3}, {'month': 'may', 'filename': 'PGE/Month05/output/', 'weekday': 6}, {'month': 'may', 'filename': 'PGE/Month05/output/', 'weekend': 7}, {'month': 'june', 'filename': 'PGE/Month06/output/', 'weekday': 3}, {'month': 'june', 'filename': 'PGE/Month06/output/', 'weekend': 5}, {'month': 'july', 'filename': 'PGE/Month07/output/', 'weekday': 1}, {'month': 'july', 'filename': 'PGE/Month07/output/', 'weekend': 3}, {'month': 'august', 'filename':

In [36]:
import json
with open('test.json', 'w') as fp:
    json.dump(descriptor, fp)

### Match Line Notation Between GeoJson and Line Power Data

In [11]:
gridlabd_scenario_path='./GRIDLABD_Results/'+scenario_name+parseCase['filename']

### Get headers of each data file overhead, underground 
file_names=["underground_lines_power_in_A", 
            "Overhead_lines_power_in_A",
            "underground_lines_power_in_B", 
            "Overhead_lines_power_in_B",
            "underground_lines_power_in_C", 
            "Overhead_lines_power_in_C",
           ]

line_df={}
lines=[]
for filename in file_names:
    line_df[filename]=pd.read_csv(gridlabd_scenario_path+filename+".csv",skiprows=list(range(0,8)),parse_dates=['# timestamp'])
    lines=lines+list(line_df[filename].keys())

### Final list of lines 
lines=list(set(lines))
lines.remove('# timestamp')

### Get lines from the geojson
geoJsonData = json.load(open('../GMLCDjangoProject/MapApp/static/MapApp/data/model2.geo.json'))

lines_in_geojson=[]
for feature in geoJsonData['features']:
    if feature["geometry"]["type"]=="LineString":
        lines_in_geojson.append(feature['properties']['name'])

lines_in_geojson=list(set(lines_in_geojson))

matched_lines=[]
for elem in lines:
    matched_lines.append(matchLineIndex(elem))    
mismatch_list=[]
for elem in matched_lines:
    if elem not in lines_in_geojson:
        mismatch_list.append(elem)


### Parse Line Data

In [33]:
scenario_number=0
for location_name in ['PGE','SCE','SDGE']:
    for scenario_name in scenario_names:
        scenario_number=scenario_number+1
        save_final_data_file=analyzed_data_path+location_name+'/scenario'+str(scenario_number)+'_lines.csv'
        parseList=getGLDParseList(gridlabd_results_path,scenario_name,location_name)

        merge_list=[]
        for parseCase in parseList:
            gridlabd_scenario_path=gridlabd_results_path+scenario_name+"/Models/"+parseCase['filename']

            overhead_power_flow_files={'power_in_a':gridlabd_scenario_path+'Overhead_lines_power_in_A.csv',
                                       'power_in_b':gridlabd_scenario_path+'Overhead_lines_power_in_B.csv',
                                       'power_in_c':gridlabd_scenario_path+'Overhead_lines_power_in_C.csv'
                                }
            overhead_power_flow_out_files={'power_out_a':gridlabd_scenario_path+'Overhead_lines_power_out_A.csv',
                                       'power_out_b':gridlabd_scenario_path+'Overhead_lines_power_out_B.csv',
                                       'power_out_c':gridlabd_scenario_path+'Overhead_lines_power_out_C.csv'
                                }

            underground_power_flow_files={'power_in_a':gridlabd_scenario_path+'underground_lines_power_in_A.csv',
                                          'power_in_b':gridlabd_scenario_path+'underground_lines_power_in_B.csv',
                                          'power_in_c':gridlabd_scenario_path+'underground_lines_power_in_C.csv'
                            }

            underground_power_flow_out_files={'power_out_a':gridlabd_scenario_path+'underground_lines_power_in_A.csv',
                                      'power_out_b':gridlabd_scenario_path+'underground_lines_power_in_B.csv',
                                      'power_out_c':gridlabd_scenario_path+'underground_lines_power_in_C.csv'
                        }
            #### PARSE GRIDLABD
            print("*********Parsing GridlabD")

            st_time=time.time()
            df,powerOverHeadData=parsePowerFlow(overhead_power_flow_files)
            df,powerUnderGroundData=parsePowerFlow(underground_power_flow_files)

            df,powerOverHeadOutData=parsePowerFlow(overhead_power_flow_out_files)
            df,powerUnderGroundOutData=parsePowerFlow(underground_power_flow_out_files)

            if 'weekend' in parseCase.keys():
                print('Parsing Weekend')
                day_of_month=parseCase['weekend']
                weekend=True
            else:
                day_of_month=parseCase['weekday']
                weekend=False

            daily_overhead_line_power=powerOverHeadData[powerOverHeadData['month']==parseCase['month']][powerOverHeadData['day_of_month']==day_of_month][["month","hour_of_day","line",
                                                                                                             "power_in_b_magnitude","power_in_c_magnitude" ,"power_in_a_magnitude",
                                                                                                             "power_in_b_Q","power_in_c_Q" ,"power_in_a_Q",
                                                                                                               "power_in_b_P","power_in_c_P" ,"power_in_a_P"]]

            daily_underground_line_power=powerUnderGroundData[powerUnderGroundData['month']==parseCase['month']][powerUnderGroundData['day_of_month']==day_of_month][["month","hour_of_day","line",
                                                                                                             "power_in_b_magnitude","power_in_c_magnitude" ,"power_in_a_magnitude"]]

            daily_overhead_line_out_power=powerOverHeadOutData[powerOverHeadOutData['month']==parseCase['month']][powerOverHeadOutData['day_of_month']==day_of_month][["month","hour_of_day","line",
                                                                                                             "power_out_b_magnitude","power_out_c_magnitude" ,"power_out_a_magnitude",
                                                                                                             "power_out_b_Q","power_out_c_Q" ,"power_out_a_Q",
                                                                                                               "power_out_b_P","power_out_c_P" ,"power_out_a_P"]]

            st_time2=time.time()
            print("Took ",st_time2-st_time)
            st_time=st_time2

    #         powerFrame=daily_overhead_line_power.append(daily_underground_line_power)
    #         powerOutFrame=daily_overhead_line_out_power.append(daily_underground_line_out_power)


            powerFrame=daily_overhead_line_power
            powerOutFrame=daily_overhead_line_out_power
            merge_test=powerFrame.merge(powerOutFrame,on=['month', 'hour_of_day','line'],how='left')


            if weekend:
                merge_test['daytype']='weekend'
            else:
                merge_test['daytype']='weekday'

            merge_test['power_flow_a']=merge_test['power_in_a_magnitude']-merge_test['power_out_a_magnitude']
            merge_test['power_flow_b']=merge_test['power_in_b_magnitude']-merge_test['power_out_b_magnitude']
            merge_test['power_flow_c']=merge_test['power_in_c_magnitude']-merge_test['power_out_c_magnitude']

            merge_test['power_flow_a_Q']=merge_test['power_in_a_Q']-merge_test['power_out_a_Q']
            merge_test['power_flow_b_Q']=merge_test['power_in_b_Q']-merge_test['power_out_b_Q']
            merge_test['power_flow_c_Q']=merge_test['power_in_c_Q']-merge_test['power_out_c_Q']


            merge_test['power_flow_a_P']=merge_test['power_in_a_P']-merge_test['power_out_a_P']
            merge_test['power_flow_b_P']=merge_test['power_in_b_P']-merge_test['power_out_b_P']
            merge_test['power_flow_c_P']=merge_test['power_in_c_P']-merge_test['power_out_c_P']





            #merge_test=merge_test.merge(daily_node_voltage,on=['month', 'hour_of_day','node'],how='left')
            merge_list.append(merge_test)




        print('Saving file to '+save_final_data_file)
        merge_test_df=pd.concat(merge_list)
        merge_test_df.reset_index(drop=True).to_csv(save_final_data_file)






*********Parsing GridlabD
Took  0.53981614112854
*********Parsing GridlabD
Parsing Weekend
Took  0.5260219573974609
*********Parsing GridlabD
Took  0.5201296806335449
*********Parsing GridlabD
Parsing Weekend
Took  0.5320758819580078
*********Parsing GridlabD
Took  0.5194952487945557
*********Parsing GridlabD
Parsing Weekend
Took  0.6296031475067139
*********Parsing GridlabD
Took  0.5998120307922363
*********Parsing GridlabD
Parsing Weekend
Took  0.7168018817901611
*********Parsing GridlabD
Took  0.6003482341766357
*********Parsing GridlabD
Parsing Weekend
Took  0.5079669952392578
*********Parsing GridlabD
Took  0.5296928882598877
*********Parsing GridlabD
Parsing Weekend
Took  0.6236171722412109
*********Parsing GridlabD
Took  0.6360478401184082
*********Parsing GridlabD
Parsing Weekend
Took  0.6361517906188965
*********Parsing GridlabD
Took  0.531919002532959
*********Parsing GridlabD
Parsing Weekend
Took  0.5525448322296143
*********Parsing GridlabD
Took  0.6461911201477051
********

### Parse Transmission Data


In [118]:
from os import listdir
import csv

def get_daytype(datetime):
    if datetime.weekday()<5:
        return "weekday"
    else: 
        return "weekend"

def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

### Get second line for object name

scenario_number=0
location_name='transmission'
for scenario_name in scenario_names:
    ### GET GLD Results to obtain info on dates.
    parseList=getGLDParseList(gridlabd_results_path,scenario_name,'PGE')
    scenario_number=scenario_number+1
    transmissionbus_dfs=[]
    transmissionline_dfs=[]
    for i in range(1,13):
        fileLocation="Month"+"%02d" % i + "/"
        transmission_folder_path=transmission_results_path+scenario_name+'/Models/griddyn_output/'+ fileLocation
        csvList=find_csv_filenames( transmission_folder_path)
        date_list=[]
        dateColumn=[]
        for pars in parseList:
            if (pars['date'].month==i):
                for hours in range(0,24):
                    dateColumn.append(datetime(pars['date'].year,pars['date'].month,pars['date'].day,)+timedelta(hours=hours))
                    
        
        for filename in csvList:
            print(filename)
            if "bus" in filename:
                name=filename.split('data.csv')
                df=pd.read_csv(transmission_folder_path+filename,skiprows=list(range(0,2)),header=None, names=
                               ['Timestamp',
                                'Vm(kV)','Va(deg)','Pg(kW)',
                                'Qg(KVar)','Pl(kW)','Ql(KVar)','link real',
                                'link reactive'
                               ])
                df=df.iloc[::6, :]
                df=df.drop([288])
#                 print(len(df),len(dateColumn))
                df['Dates']=dateColumn
                df['node']=name[0]
                
#               Time, voltage, angle, generation real, generation reactive,
#               load real, load reactive,link real,link reactive

                df['voltage_magnitude']=df['Vm(kV)']
                df['voltage_angle']=df['Va(deg)']
                df['active_power']=df['Pl(kW)']
                df['reactive_power']=df['Ql(KVar)']

                df['month'] = df['Dates'].map(lambda x: monthMapper(x.month))
                df['hour_of_day'] = df['Dates'].map(lambda x: x.hour)
                df['day_of_month'] = df['Dates'].map(lambda x: x.day)
                df['daytype'] = df['Dates'].map(lambda x: get_daytype(x))
                df['timezone']= df['Dates'].map(lambda x: x.tz)
                df.drop(['Dates','Vm(kV)','Va(deg)','Pl(kW)','Ql(KVar)','Pg(kW)',
                                'Qg(KVar)','link real','link reactive'], axis=1,inplace=True)
                transmissionbus_dfs.append(df)
            elif ("link" in filename):
                seperate='data'
                if ("tx" in filename):
                    continue
                if ("circuit" in filename):
                    seperate='circuit'
                    
                name=filename.split(seperate)
                df=pd.read_csv(transmission_folder_path+filename,skiprows=list(range(0,2)),header=None, 
                               names= ['Timestamp','Pfrom(kW)', 'Qfrom(KVar)','Pto(kW)', 'Qto(kVar)'])
                df=df.iloc[::6, :]
                df=df.drop([288])
                df['Dates']=dateColumn
                df['node']=name[0]
#                 df=pd.read_csv(transmission_folder_path+filename,skiprows=list(range(0,3)),parse_dates=['Dates'])
#                 df['Dates'] = df['Dates'].map(lambda x: x.tz_localize('UTC').tz_convert('US/Pacific'))

                df['active_power_from']=df['Pfrom(kW)']
                df['active_power_to']=df['Pto(kW)']
                df['reactive_power_from']=df['Qfrom(KVar)']
                df['reactive_power_to']=df['Qto(kVar)']


                df['month'] = df['Dates'].map(lambda x: monthMapper(x.month))
                df['hour_of_day'] = df['Dates'].map(lambda x: x.hour)
                df['day_of_month'] = df['Dates'].map(lambda x: x.day)
                df['daytype'] = df['Dates'].map(lambda x: get_daytype(x))
                df['timezone']= df['Dates'].map(lambda x: x.tz)
                df.drop(['Dates','Pfrom(kW)','Pto(kW)','Qfrom(KVar)','Qto(kVar)'], axis=1,inplace=True)
                df['line']=name[0]
                transmissionline_dfs.append(df)

    transmission_lines=pd.concat(transmissionline_dfs)
    transmission_nodes=pd.concat(transmissionbus_dfs)
    transmission_lines.drop(['Timestamp'],axis=1,inplace=True)
    transmission_nodes.drop(['Timestamp'],axis=1,inplace=True)

    transmission_lines.to_csv('../GMLCDjangoProject/MapApp/static/MapApp/data/transmission/scenario'+str(scenario_number) +'_lines.csv')
    transmission_nodes.to_csv('../GMLCDjangoProject/MapApp/static/MapApp/data/transmission/scenario'+str(scenario_number) +'.csv')

WECCbus157data.csv
WECCbus54data.csv
WECCbus57data.csv
WECClink156-157data.csv
WECClink157-161data.csv
WECClink42-57data.csv
WECClink50-57data.csv
WECClink54-51tx1data.csv
WECClink54-51tx2data.csv
WECClink54-57circuit1data.csv
WECClink54-57circuit2data.csv
WECClink57-58data.csv
WECCbus157data.csv
WECCbus54data.csv
WECCbus57data.csv
WECClink156-157data.csv
WECClink157-161data.csv
WECClink42-57data.csv
WECClink50-57data.csv
WECClink54-51tx1data.csv
WECClink54-51tx2data.csv
WECClink54-57circuit1data.csv
WECClink54-57circuit2data.csv
WECClink57-58data.csv
WECCbus157data.csv
WECCbus54data.csv
WECCbus57data.csv
WECClink156-157data.csv
WECClink157-161data.csv
WECClink42-57data.csv
WECClink50-57data.csv
WECClink54-51tx1data.csv
WECClink54-51tx2data.csv
WECClink54-57circuit1data.csv
WECClink54-57circuit2data.csv
WECClink57-58data.csv
WECCbus157data.csv
WECCbus54data.csv
WECCbus57data.csv
WECClink156-157data.csv
WECClink157-161data.csv
WECClink42-57data.csv
WECClink50-57data.csv
WECClink54-51tx1

### Locations for transmission nodes and lines

In [116]:
transmission_lines

,node,active_power_from,active_power_to,reactive_power_from,reactive_power_to,month,hour_of_day,day_of_month,daytype,timezone,line
0,WECClink156-157,2.342316,-2.340719,-0.005199,0.101551,january,0,1,weekday,None,WECClink156-157
6,WECClink156-157,2.116236,-2.114915,-0.373581,0.453265,january,1,1,weekday,None,WECClink156-157
12,WECClink156-157,2.080880,-2.079602,-0.374422,0.451512,january,2,1,weekday,None,WECClink156-157
18,WECClink156-157,2.070539,-2.069273,-0.374645,0.450985,january,3,1,weekday,None,WECClink156-157
24,WECClink156-157,2.068648,-2.067385,-0.374653,0.450855,january,4,1,weekday,None,WECClink156-157
30,WECClink156-157,2.093156,-2.091863,-0.374117,0.452101,january,5,1,weekday,None,WECClink156-157
36,WECClink156-157,2.165864,-2.164482,-0.372198,0.455598,january,6,1,weekday,None,WECClink156-157
42,WECClink156-157,2.290151,-2.288608,-0.367813,0.460912,january,7,1,weekday,None,WECClink156-157
48,WECClink156-157,2.279723,-2.278194,-0.367460,0.459705,january,8,1,weekday,None,WECClink156-157
54,WECClink156-157,1.800173,-1.799212,-0.374362,0.432390,january,9,1,weekday,None,WECClink156-157


In [119]:
print(transmission_lines['node'].unique())
print(transmission_nodes['node'].unique())

['WECClink156-157' 'WECClink157-161' 'WECClink42-57' 'WECClink50-57'
 'WECClink54-57' 'WECClink57-58']
['WECCbus157' 'WECCbus54' 'WECCbus57']


### Shift regions for SCE and SDGE

In [181]:
import json
from pprint import pprint

endpoint_list=['node','load','substations','model.geo']
### MOVING FROM  35.416667,-119.05
### TO Santa Ana 33.7394197,-117.9055612
### TO San Diego 32.7012145,-117.110277

delta_longitude=-117.9055612+119.05
delta_latitude=33.7394197-35.416667
region="sce"

for elem_name in endpoint_list:
    print("Parsing "+elem_name)
    data = json.load(open('../GMLCDjangoProject/MapApp/static/MapApp/data/pge/endpoints/'+elem_name+'.json'))
    if elem_name=='model.geo':
        for elem in data['features']:
            if elem['geometry']['type']=="Point":
                elem['geometry']['coordinates'][0]=elem['geometry']['coordinates'][0]+delta_longitude
                elem['geometry']['coordinates'][1]=elem['geometry']['coordinates'][1]+delta_latitude
            elif elem['geometry']['type']=="LineString":
                for elemcoor in elem['geometry']['coordinates']:
                    elemcoor[0]=elemcoor[0]+delta_longitude
                    elemcoor[1]=elemcoor[1]+delta_latitude
        
        with open('../GMLCDjangoProject/MapApp/static/MapApp/data/'+region+'/endpoints/'+elem_name+'.json', 'w') as fp:
            json.dump(data,fp)
    
    else:
        for elem in data:
            if 'latitude' in elem.keys():
                elem['longitude']=float(elem['longitude'])+delta_longitude
                elem['latitude']=float(elem['latitude'])+delta_latitude
        with open('../GMLCDjangoProject/MapApp/static/MapApp/data/'+region+'/endpoints/'+elem_name+'.json', 'w') as fp:
            json.dump(data,fp)

### TO San Diego 32.7012145,-117.110277
            
delta_longitude=-117.110277+119.05
delta_latitude=32.7012145-35.416667
region="sdge"

for elem_name in endpoint_list:
    print("Parsing "+elem_name)
    data = json.load(open('../GMLCDjangoProject/MapApp/static/MapApp/data/pge/endpoints/'+elem_name+'.json'))
    if elem_name=='model.geo':
        for elem in data['features']:
            if elem['geometry']['type']=="Point":
                elem['geometry']['coordinates'][0]=elem['geometry']['coordinates'][0]+delta_longitude
                elem['geometry']['coordinates'][1]=elem['geometry']['coordinates'][1]+delta_latitude
            elif elem['geometry']['type']=="LineString":
                for elemcoor in elem['geometry']['coordinates']:
                    elemcoor[0]=elemcoor[0]+delta_longitude
                    elemcoor[1]=elemcoor[1]+delta_latitude
        
        with open('../GMLCDjangoProject/MapApp/static/MapApp/data/'+region+'/endpoints/'+elem_name+'.json', 'w') as fp:
            json.dump(data,fp)
    
    else:
        for elem in data:
            if 'latitude' in elem.keys():
                elem['longitude']=float(elem['longitude'])+delta_longitude
                elem['latitude']=float(elem['latitude'])+delta_latitude
        with open('../GMLCDjangoProject/MapApp/static/MapApp/data/'+region+'/endpoints/'+elem_name+'.json', 'w') as fp:
            json.dump(data,fp)


Parsing node
Parsing load
Parsing substations
Parsing model.geo
Parsing node
Parsing load
Parsing substations
Parsing model.geo


In [19]:
scenario_names[0]

'BC'